# Instructions in Munipack
* New Project
* Title With Date e.g. 20161117
* Add Frames, all of one filter and exposure, repeat for multiple filters and exposures
* Edit Project Settings
    * Star Detection - FWHM = 6.50
    * Matching - Algorithm for Dense Fields
* Covert Input Files to Working Format
* Photometry
* Matching - Select Best Field
* Select Both Light Curve and Find Variables
* Find Variables
    * Select Aperture #4
    * Export varfind data
* Light Curve
    * Select Heliocentric Correction
    * Select Ensemble Photometry
    Select All stars on the reference frame
    * SELECT COMP STARS - Bright and unsaturated, close by
    * Select Aperture #4
    * View Table
    * File -> Save (as date)
    * Hash the first two lines of that file
    
* Go To folder and create folder lightcurves in varfind_date

* Enter date and Run Script

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from dateutil.parser import parse
from scipy import signal

date = str(20161117)

folderpath = './varfind_'+date+'/'
#Hash the first two lines of this file
comp_filename = date + '.txt'
img_filename = 'varfind.fts'
vardata = np.loadtxt(folderpath + comp_filename, delimiter=" ")

image_file = fits.open(folderpath+img_filename)
image_data = image_file[0].data
image_header = image_file[0].header

JD = image_header["JD"]
exposure = image_header["EXPTIME"]
filter_type = image_header["FILTER"]
#date = parse(image_header["DATE-OBS"]).date()
location = '('+image_header["RA"]+', '+image_header["DEC"]+')'

#Creating Arrays for Data

varJD = vardata[:, 0]
varmag_JD = vardata[:, 1::2]
varmag_star = np.transpose(varmag_JD)
varmagerror_JD = vardata[:, ::2]
varmagerror_JD = varmagerror_JD[:, 1:]

image_file.close()

star_period_frames = [(0, 0, 0)]

for n in range(0, len(varmag_star)):
    mag = varmag_star[n].tolist()
    nJD = varJD.tolist()
    for idx, val in enumerate(reversed(mag)):
        if(val >= 99):
            mag.pop((len(varmag_star[n]) - 1) - idx)
            nJD.pop((len(varmag_star[n]) - 1) - idx)

    fig = plt.figure(figsize=(20,8))
    plt.scatter(nJD, mag)
    plt.xlabel(r'${\rm JD}$', fontsize=18)
    plt.ylabel(r'${\rm Mag}$', fontsize=18)
    plt.xlim(np.min(nJD)-0.01, np.max(nJD)+0.01)
    plt.ylim((np.min(mag)-0.3, np.max(mag)+0.3))
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date), fontsize=18)
    fig.savefig('./varfind_'+date+'/lightcurves/'+'lc_' + str(n) + '.png')
    plt.close()
    
    fs = 1/exposure
    f, Pxx_den = signal.periodogram(mag, fs)
    star_period_frames.append((n, np.amax(Pxx_den), len(mag)))
    fig = plt.figure()
    plt.semilogy(f, Pxx_den)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('Linear spectrum [V RMS]')
    fig.suptitle('NGC 2180: Star ' + str(n) + '| Date: ' + str(date) + ' Periodogram', fontsize=18)
    fig.savefig('./varfind_'+date+'/lightcurves/'+'lc_' + str(n) + '_pdgm.png')
    plt.close()
    
period_star_frames = sorted(star_period_frames, key=lambda tup: tup[1], reverse = True)
with open('./varfind_'+date+'/variablestars.txt', 'w') as fp:
    fp.write('\n'.join('%s, %s, %s' % x for x in  period_star_frames))

[[  2.45771065e+06  -2.40260000e+00   1.48000000e-03 ...,   9.99999000e+00
    3.14365000e+00   1.43500000e-02]
 [  2.45771066e+06  -2.39936000e+00   1.42000000e-03 ...,   9.99999000e+00
    3.16831000e+00   1.26100000e-02]
 [  2.45771066e+06  -2.40667000e+00   1.51000000e-03 ...,   9.99999000e+00
    3.21387000e+00   1.32700000e-02]
 ..., 
 [  2.45771087e+06  -2.37574000e+00   1.27000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -2.38670000e+00   1.25000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]
 [  2.45771088e+06  -2.39004000e+00   1.27000000e-03 ...,   9.99999000e+00
    9.99999900e+01   9.99999000e+00]]


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\ticker.py:2210: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  "Data has no positive values, and therefore cannot be "


[(231, 3.3258673897500053, 12), (212, 2.1249970831230485, 97), (312, 0.99401815885856748, 69), (162, 0.94583737625914632, 81), (377, 0.71139522100847252, 44), (250, 0.70563280737221135, 87), (222, 0.6662350365024674, 95), (287, 0.60382179205523556, 88), (329, 0.58953889459926256, 62), (260, 0.56680405458456706, 91), (203, 0.5642878949970126, 100), (396, 0.53973583271203651, 28), (80, 0.53301447210808128, 78), (322, 0.5003902391087699, 85), (183, 0.4859024305898883, 100), (298, 0.47588381119465356, 81), (264, 0.46757853093993212, 83), (274, 0.46377660531486742, 79), (283, 0.45479976769555885, 80), (240, 0.44493064506031615, 95), (220, 0.444555864816645, 99), (351, 0.44444019890483266, 67), (262, 0.44280585169505282, 88), (161, 0.43720617098364145, 97), (142, 0.42724011893409125, 98), (428, 0.40314007809090802, 33), (303, 0.39124682540233902, 69), (214, 0.38957292739195959, 98), (374, 0.38860651421346853, 42), (280, 0.38735269075000012, 12), (289, 0.38695379623206205, 62), (435, 0.386377

### Look for the highest Periods, those stars are probably variable, not ones with low frame count